In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime
import os
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from google.cloud import bigquery

In [2]:
#La ruta al archivo JSON está en una variable de entorno del sistema operativo
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]

'C://pythonproy//Project-74ca222cd7de.json'

## 1. The following spreadsheet contains call info from a telesales team, read it using Python

In [3]:
# use creds to create a client to interact with the Google Drive API
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name(os.environ["GOOGLE_APPLICATION_CREDENTIALS"], scope)
client = gspread.authorize(creds)

# Find a workbook by name and open the first sheet
# Make sure you use the right name here.
sheet = client.open("test_data").sheet1

# Extract and print all of the values
list_of_hashes = sheet.get_all_records()
df = pd.DataFrame(list_of_hashes)
df = df.replace(r'^\s*$', np.nan, regex=True)
df.head(5)

,campaign,category,day_of_month,disposition,month,quarter_hour,year
0,prospecting,NaN,1,Email,December,NaN,2017
1,outbound,NaN,1,Email,December,NaN,2017
2,prospecting,NaN,1,Email,December,NaN,2017
3,prospecting,NaN,1,Email,December,NaN,2017
4,prospecting,NaN,1,Email,December,NaN,2017


In [4]:
#Cargando el CSV en un dataframe
#df = pd.read_csv("test_data.csv", delimiter=",")
#df.head(5)

## 2. As you can see, the data has time information broken down in different columns. Consolidate all time information into a timestamp that is localized in MST (Mountain Standard Time) and then build another column with timestamps in UTC. 
Note: If quarter_hour not provided choose 00:00 as the time to build the timestamp.

In [5]:
#Revisando cuantos registros tienen vacío el campo quarter_hour
df_test = df[df['quarter_hour'].isnull() == True]
df_test.shape[0] #1838 campos vacíos

1838

In [6]:
#Revisando los diferentes valores que tiene el campo quarter_hour, y agrupando para contarlos
df_test = df.groupby(['quarter_hour']).size().reset_index(name='counts')
df_test.head(5)

,quarter_hour,counts
0,10:00,39
1,10:15,44
2,10:30,48
3,10:45,48
4,11:00,48


In [7]:
#Llenando los quarter_hour con 00:00
df['quarter_hour'] = df.quarter_hour.fillna('00:00')
df.head(5)

,campaign,category,day_of_month,disposition,month,quarter_hour,year
0,prospecting,NaN,1,Email,December,00:00,2017
1,outbound,NaN,1,Email,December,00:00,2017
2,prospecting,NaN,1,Email,December,00:00,2017
3,prospecting,NaN,1,Email,December,00:00,2017
4,prospecting,NaN,1,Email,December,00:00,2017


In [8]:
#Revisando cuantos registros tienen vacío el campo quarter_hour
df_test = df[df['quarter_hour'].isnull() == True]
df_test.shape[0] #ningun campo vacío

0

In [9]:
#Revisando cuantos registros quedaron con 00:00
df_test = df[df['quarter_hour'] == '00:00']
df_test.shape[0] #1838 campos

1838

In [10]:
#Concatenando toda la fecha en un sólo campo 
df['fulldate'] = df['year'].map(str) +'-'+ df['month'] +'-'+ df['day_of_month'].map(str) +' '+ df['quarter_hour']
df['fulldate'] = df['fulldate'].apply(lambda x: dt.datetime.strptime(x,'%Y-%B-%d %H:%M'))
df.head(5)

,campaign,category,day_of_month,disposition,month,quarter_hour,year,fulldate
0,prospecting,NaN,1,Email,December,00:00,2017,2017-12-01
1,outbound,NaN,1,Email,December,00:00,2017,2017-12-01
2,prospecting,NaN,1,Email,December,00:00,2017,2017-12-01
3,prospecting,NaN,1,Email,December,00:00,2017,2017-12-01
4,prospecting,NaN,1,Email,December,00:00,2017,2017-12-01


In [11]:
def converter_MST(row):
    return row['fulldate'].tz_localize('US/Eastern').tz_convert('US/Mountain')

#Creando la fecha con Timezone MST
df['MST_Time'] = df.apply(converter_MST, axis=1)
df.head(5)

,campaign,category,day_of_month,disposition,month,quarter_hour,year,fulldate,MST_Time
0,prospecting,NaN,1,Email,December,00:00,2017,2017-12-01,2017-11-30 22:00:00-07:00
1,outbound,NaN,1,Email,December,00:00,2017,2017-12-01,2017-11-30 22:00:00-07:00
2,prospecting,NaN,1,Email,December,00:00,2017,2017-12-01,2017-11-30 22:00:00-07:00
3,prospecting,NaN,1,Email,December,00:00,2017,2017-12-01,2017-11-30 22:00:00-07:00
4,prospecting,NaN,1,Email,December,00:00,2017,2017-12-01,2017-11-30 22:00:00-07:00


In [12]:
def converter_UTC(row):
    return row['fulldate'].tz_localize('US/Eastern').tz_convert('UTC')

#Creando la fecha con Timezone UTC
df['UTC_Time'] = df.apply(converter_UTC, axis=1)

df.head(5)

,campaign,category,day_of_month,disposition,month,quarter_hour,year,fulldate,MST_Time,UTC_Time
0,prospecting,NaN,1,Email,December,00:00,2017,2017-12-01,2017-11-30 22:00:00-07:00,2017-12-01 05:00:00+00:00
1,outbound,NaN,1,Email,December,00:00,2017,2017-12-01,2017-11-30 22:00:00-07:00,2017-12-01 05:00:00+00:00
2,prospecting,NaN,1,Email,December,00:00,2017,2017-12-01,2017-11-30 22:00:00-07:00,2017-12-01 05:00:00+00:00
3,prospecting,NaN,1,Email,December,00:00,2017,2017-12-01,2017-11-30 22:00:00-07:00,2017-12-01 05:00:00+00:00
4,prospecting,NaN,1,Email,December,00:00,2017,2017-12-01,2017-11-30 22:00:00-07:00,2017-12-01 05:00:00+00:00


In [13]:
#Verificando los tipos de datos
df.dtypes

campaign                             object
category                             object
day_of_month                          int64
disposition                          object
month                                object
quarter_hour                         object
year                                  int64
fulldate                     datetime64[ns]
MST_Time        datetime64[ns, US/Mountain]
UTC_Time                datetime64[ns, UTC]
dtype: object

## 3. Save the data in the database engine that you want.

In [15]:
client = bigquery.Client()
dataset_ref = client.dataset('test_data')
table_ref = dataset_ref.table('calls')

client.load_table_from_dataframe(df, table_ref).result()

## 4. We need to have this process automatized, propose a solution.

We can adjust this code to load only the new rows, if needed, and then we can migrate this code to a single file (something like etl.py).

Then we can create an scheduled task to run it periodically.